In [ ]:
"""
Inference Script
"""
import json
import os
import torch
import tqdm
from shutil import copyfile


# Detectron imports|
from detectron2.engine import launch
from detectron2.data import build_detection_test_loader, build_detection_train_loader, MetadataCatalog

# Project imports
import core.datasets.metadata as metadata

from core.setup import setup_config, setup_arg_parser
from offline_evaluation import compute_average_precision, compute_probabilistic_metrics, compute_calibration_errors
from probabilistic_inference.probabilistic_inference import build_predictor
from probabilistic_inference.inference_utils import instances_to_json

from baal.active import FileDataset, ActiveLearningDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, verify_results

In [ ]:
##setup inference args, this also contains all the training args
arg_parser = setup_arg_parser()
args = arg_parser.parse_args("")
# Support single gpu inference only.
args.num_gpus = 1
args.dataset_dir = '/public-dataset/BDD/bdd100k'
args.test_dataset = 'bdd_val'
args.config_file = '/home/jefferson.keh/AL_CVPR/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml'
args.inference_config = '/home/jefferson.keh/AL_CVPR/src/configs/Inference/bayes_od_mc_dropout.yaml'
print("Command Line Args:", args)

In [ ]:
cfg = setup_config(args, random_seed=1234)

In [ ]:
from detectron2.data import get_detection_dataset_dicts
from detectron2.data import build_detection_train_loader, build_detection_test_loader

In [ ]:
dataset = get_detection_dataset_dicts(        
    cfg.DATASETS.TRAIN,
    filter_empty=cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS,
    min_keypoints=cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE
    if cfg.MODEL.KEYPOINT_ON
    else 0,
    proposal_files=cfg.DATASETS.PROPOSAL_FILES_TRAIN if cfg.MODEL.LOAD_PROPOSALS else None,
)

In [ ]:
from torch.utils.data import Subset
import numpy as np

np.random.seed(0)
indices = np.random.choice(len(dataset), 2000, replace=False)
subsample_dataset = Subset(dataset, indices)

In [ ]:
print(indices)

In [ ]:
dataset = ActiveLearningDataset(dataset)
subsample_dataset = ActiveLearningDataset(subsample_dataset)

In [ ]:
print(dataset.pool.__len__())
print(subsample_dataset.pool.__len__())

In [ ]:
print(dataset.pool[40764]['file_name'])
print(subsample_dataset.pool[0]['file_name'])

In [ ]:
print(subsample_dataset.__len__())
print(subsample_dataset.pool.__len__())
print(subsample_dataset.pool[0]['file_name'])
subsample_dataset.label(0)
print(subsample_dataset.__len__())
print(subsample_dataset.pool.__len__())
print(subsample_dataset.pool[0]['file_name'])

In [ ]:
from detectron2.data import DatasetMapper

mapper = DatasetMapper(cfg, False)
data_loader = build_detection_test_loader(dataset=subsample_dataset.pool, mapper=mapper)

In [ ]:
data_loader_iter = iter(data_loader)

In [ ]:
print(next(data_loader_iter))

# Other stuff I was just testing

In [ ]:
torch.manual_seed(0)
torch.rand(5)

In [ ]:
torch.rand(5)

In [ ]:
from detectron2.data import build_detection_train_loader
from detectron2.data.samplers import TrainingSampler

In [ ]:
dataloader1 = build_detection_train_loader(cfg)

sampler2 = TrainingSampler(size=len(dataloader1.dataset.dataset), seed=1234)

dataloader2 = build_detection_train_loader(cfg, sampler=sampler2)

sampler3 = TrainingSampler(size=4, seed=1234)

dataloader3 = build_detection_train_loader(cfg, sampler=sampler3)


In [ ]:
# To show that each is a different object
print("DataLoader 1 Memory Address: ")
print(dataloader1)
print("DataLoader 2 Memory Address: ")
print(dataloader2)
print("DataLoader 3 Memory Address: ")
print(dataloader3)
print("\n")

print("Dataset Lengths")
print("DataLoader 1: ", len(dataloader1.dataset.dataset))
print("DataLoader 2: ", len(dataloader2.dataset.dataset))
print("DataLoader 3: ", len(dataloader3.dataset.dataset))
print("\n")

print("Sampler Random Seeds")
print("DataLoader 1: ", dataloader1.dataset.sampler._seed)
print("DataLoader 2: ", dataloader2.dataset.sampler._seed)
print("DataLoader 3: ", dataloader3.dataset.sampler._seed)
print("\n")

print("Sampler Size")
print("DataLoader 1: ", dataloader1.dataset.sampler._size)
print("DataLoader 2: ", dataloader2.dataset.sampler._size)
print("DataLoader 3: ", dataloader3.dataset.sampler._size)
print("\n")

In [ ]:
print(torch.initial_seed())

In [ ]:
dataset = dataloader1.dataset.dataset

In [ ]:
dataset.__len__()

In [ ]:
dataloader2.dataset.dataset = dataset